In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

# from datasets import load_dataset

# dataset = load_dataset("yelp_review_full")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
import pickle
# with open('yelp_dataset.pkl', 'wb') as file:
#     pickle.dump(dataset, file)

In [3]:
with open('dbpedia.pkl', 'rb') as file:
    dataset = pickle.load(file)

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['label', 'title', 'content'],
        num_rows: 560000
    })
    test: Dataset({
        features: ['label', 'title', 'content'],
        num_rows: 70000
    })
})


In [5]:
train_dataset = dataset['train']
test_dataset = dataset['test']

# Convert train and test datasets to arrays
train_data_title = train_dataset['title']
train_data_content = train_dataset['content']
train_labels = train_dataset['label']
test_data_title = test_dataset['title']
test_data_content = test_dataset['content']
test_labels = test_dataset['label']

In [6]:
# print(train_data_title[0])
# print(train_data_content[0])
# print(train_labels[0])


In [7]:
# train_data = train_data_title + train_data_content
# test_data = test_data_title + test_data_content

train_data = [None] * len(train_data_title)
for i in range(len(train_data_title)):
    train_data[i] = train_data_title[i] + train_data_content[i]

test_data = [None] * len(test_data_title)
for i in range(len(test_data_title)):
    test_data[i] = test_data_title[i] + test_data_content[i]

## Preprocess

In [8]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/revanthgundam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/revanthgundam/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
import sys  
!{sys.executable} -m pip install contractions
import contractions

In [10]:
# import re
# import nltk
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
# nltk.download('stopwords')
# nltk.download('punkt')

# def custom_contractions_fix(text):
#     # Define custom contractions to expand
#     contractions_dict = {
#         "don't": "do not",
#         "doesn't": "does not",
#         "didn't": "did not",
#         # Add more contractions as needed
#     }
    
#     # Use a regular expression to find and replace contractions
#     contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()))
    
#     def replace(match):
#         return contractions_dict[match.group(0)]
    
#     expanded_text = contractions_re.sub(replace, text)
#     return expanded_text


# def tokenize_text(text):
    
#     # if there is a word between () then write it twice
#     text = re.sub(r'\((.*?)\)', r'\1 \1', text)
    
#     # Apply custom contractions expansion
#     text = custom_contractions_fix(text)
    
#     # Replace hyphens with spaces
#     text = text.replace('-', ' ')
    
#     tokens = nltk.word_tokenize(text)
    
#     # Add an extra occurrence for all-uppercase words with more than one letter
#     temp = [word for word in tokens if len(word) > 1 and word.isupper()]
#     tokens.extend(temp)
    
#     # Convert to lowercase
#     tokens = [w.lower() for w in tokens]
    
#     # Remove periods
#     tokens = [w.replace('.', '') for w in tokens]
    
#     # Remove punctuation and stopwords
#     stop_words = set(stopwords.words("english"))
#     tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    
#     return tokens



In [11]:
# # print(tokenize_text(train_data[10]))

# tokens = []
# print(len(train_data))

# for i in range(len(train_data)):
#     tokens.append(tokenize_text(train_data[i]))
#     if i % 1000 == 0:
#         print(i)

# # save to pkl file
# with open('dbpedia_train_tokens_no_stop.pkl', 'wb') as file:
#     pickle.dump(tokens, file)

In [12]:
# import pickle
# from collections import Counter

# # Load the tokenized data
# with open('dbpedia_train_tokens_no_stop.pkl', 'rb') as file:
#     tokens = pickle.load(file)

# # Define the frequency cutoff threshold
# frequency_cutoff = 30

# # Count word frequencies
# word_counts = Counter(word for tokens_list in tokens for word in tokens_list)

# # Filter out words with counts less than the threshold
# filtered_tokens = [[word for word in tokens_list if word_counts[word] >= frequency_cutoff] for tokens_list in tokens]

# # Save the filtered tokens to a new file
# with open('dbpedia_train_tokens_filtered.pkl', 'wb') as file:
#     pickle.dump(filtered_tokens, file)

In [13]:
# tokens_test = []
# print(len(test_data))

# for i in range(len(test_data)):
#     tokens_test.append(tokenize_text(test_data[i]))
#     if i % 1000 == 0:
#         print(i)

# with open('dbpedia_test_tokens_no_stop.pkl', 'wb') as file:
#     pickle.dump(tokens_test, file)

In [14]:
# import pickle
# from collections import Counter

# # Load the tokenized data
# with open('dbpedia_test_tokens_no_stop.pkl', 'rb') as file:
#     tokens = pickle.load(file)

# # Define the frequency cutoff threshold
# frequency_cutoff = 30

# # Count word frequencies
# word_counts = Counter(word for tokens_list in tokens for word in tokens_list)

# # Filter out words with counts less than the threshold
# filtered_tokens = [[word for word in tokens_list if word_counts[word] >= frequency_cutoff] for tokens_list in tokens]

# # Save the filtered tokens to a new file
# with open('dbpedia_test_tokens_filtered.pkl', 'wb') as file:
#     pickle.dump(filtered_tokens, file)

In [15]:
import numpy as np

train_tokens = []
with open('dbpedia_train_tokens_filtered.pkl', 'rb') as file:
    train_tokens = pickle.load(file)

test_tokens = []
with open('dbpedia_test_tokens_filtered.pkl', 'rb') as file:
    test_tokens = pickle.load(file)
    
print("Loaded tokens")



Loaded tokens


In [16]:
# in train data replace every 1000th word with UNK randomly

import random

for i in range(len(train_tokens)):
    for j in range(len(train_tokens[i])):
        if random.randint(1, 1000) == 1:
            train_tokens[i][j] = 'UNK'
        

In [17]:
# Build the BoW representation manually
# Create a vocabulary by collecting unique words from the training data
vocab = set()
for tokens in train_tokens:
    vocab.update(tokens)

# Create a dictionary to map words to indices in the vocabulary
vocab_dict = {}
for i, word in enumerate(vocab):
    vocab_dict[word] = i

print(len(vocab_dict))
# Initialize BoW matrices for training and testing data


36315


In [18]:
# randomize the order of the training data and the testing data along with their labels with a specific seed
import random

random_seed = 42

train_data = []
new_train_labels = []

test_data = []
new_test_labels = []

for i in range(len(train_tokens)):
    train_data.append(train_tokens[i])
    new_train_labels.append(train_labels[i])

for i in range(len(test_tokens)):
    test_data.append(test_tokens[i])
    new_test_labels.append(test_labels[i])

# shuffle the data
random.seed(random_seed)
temp = list(zip(train_data, new_train_labels))
random.shuffle(temp)
train_data, train_labels = zip(*temp)

random.seed(random_seed)
temp = list(zip(test_data, new_test_labels))
random.shuffle(temp)
test_data, test_labels = zip(*temp)

# convert them back to the format of train_tokens and test_tokens
train_tokens = list(train_data)
train_labels = list(train_labels)
test_tokens = list(test_data)
test_labels = list(test_labels)


In [19]:
import torch
import torch.nn as nn
import torch.optim as optim

num_epochs = 4
learning_rate = 0.01
batch_size = 10000

class BoWClassifier(nn.Module):
    def __init__(self, input_size, output_size):
        super(BoWClassifier, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        # Pass the input through the linear layer
        out = self.linear(x)
        return out
    
# Define the model
input_size = len(vocab)  # Input size is the size of the vocabulary
output_size = 14  # Output size is 13 dimensions

model = BoWClassifier(input_size, output_size).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [20]:
import numpy as np
from collections import Counter
from math import log

def calculate_tfidf(tokens, vocab_dict):

    n_docs = len(tokens)
    vocab_size = len(vocab_dict)
    
    # Calculate document frequencies (DF) for each word
    doc_freq = Counter()
    for doc in tokens:
        doc_freq.update(set(doc))
    
    # Precompute IDF values
    idf_dict = {}
    for word, count in doc_freq.items():
        idf_dict[word] = log(n_docs / (count + 1))  # Add 1 to avoid division by zero
    
    tfidf_matrix = np.zeros((n_docs, vocab_size))
    
    for i, doc in enumerate(tokens):
        # if i % 1000 == 0:
        #     print(i)
        
        total_words_in_doc = len(doc)
        term_freq = Counter(doc)  # Calculate term frequency (TF) for the document
        
        for word, tf in term_freq.items():
            if word in vocab_dict:
                tfidf_matrix[i][vocab_dict[word]] = (tf / total_words_in_doc) * idf_dict[word]
            else:
                # Use 'UNK' if the word is not in the vocabulary
                tfidf_matrix[i][vocab_dict['UNK']] = (tf / total_words_in_doc) * idf_dict[word]
    
    return tfidf_matrix

In [21]:
# Training loop
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i in range(0, len(train_tokens), batch_size):
        
        inputs = train_tokens[i:i + batch_size]
        # print(len(inputs))
        # print(len(vocab))
        
        # make bow vector for inputs
        # bow = torch.zeros((batch_size, len(vocab)), dtype=torch.float32)
        # # print(bow.shape)
        # for j in range(batch_size):
        #     for token in inputs[j]:

        #         bow[j][vocab_dict[token]] += 1

        bow = calculate_tfidf(inputs, vocab_dict)

        
        # convert bow to tensor
        inputs = torch.tensor(bow, dtype=torch.float32).to(device)
        
        labels = train_labels[i:i + batch_size]  # Make sure to have train_labels defined
        # Convert labels to LongTensors
        labels = torch.tensor(labels, dtype=torch.long).to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Calculate the loss
        loss = criterion(outputs, labels)

        # Backpropagation
        loss.backward()

        # Update weights
        optimizer.step()

        # calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()


        # Print the loss for this batch if needed
        print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{i//batch_size+1}], Loss: {loss.item()}')

    print(f'Accuracy of the network on the {total} train inputs: {100 * correct / total} %')

print('Training finished')

Epoch [1/4], Batch [1], Loss: 2.6389148235321045
Epoch [1/4], Batch [2], Loss: 2.588900566101074
Epoch [1/4], Batch [3], Loss: 2.5391645431518555
Epoch [1/4], Batch [4], Loss: 2.4892818927764893
Epoch [1/4], Batch [5], Loss: 2.439829111099243
Epoch [1/4], Batch [6], Loss: 2.3911490440368652
Epoch [1/4], Batch [7], Loss: 2.3406126499176025
Epoch [1/4], Batch [8], Loss: 2.2928597927093506
Epoch [1/4], Batch [9], Loss: 2.2468421459198
Epoch [1/4], Batch [10], Loss: 2.1975719928741455
Epoch [1/4], Batch [11], Loss: 2.1528661251068115
Epoch [1/4], Batch [12], Loss: 2.1045045852661133
Epoch [1/4], Batch [13], Loss: 2.0598368644714355
Epoch [1/4], Batch [14], Loss: 2.015774726867676
Epoch [1/4], Batch [15], Loss: 1.9716567993164062
Epoch [1/4], Batch [16], Loss: 1.9302722215652466
Epoch [1/4], Batch [17], Loss: 1.881532073020935
Epoch [1/4], Batch [18], Loss: 1.8424574136734009
Epoch [1/4], Batch [19], Loss: 1.8021113872528076
Epoch [1/4], Batch [20], Loss: 1.7588152885437012
Epoch [1/4], Bat

In [22]:
# save the above model in its entirety in pkl file
torch.save(model, 'bow_model_dbpedia.pkl')

# load the model
model = torch.load('bow_model_dbpedia.pkl')

In [23]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, classification_report

# Test the model
with torch.no_grad():
    total = 0
    correct = 0
    predictions = []
    true_labels = []

    for i in range(0, len(test_tokens), batch_size):
        inputs = test_tokens[i:i + batch_size]
        # bow = torch.zeros((batch_size, len(vocab)), dtype=torch.float32)

        # for j in range(batch_size):
        #     for token in inputs[j]:
        #         if token in vocab_dict:
        #             bow[j][vocab_dict[token]] += 1
        #         else:
        #             bow[j][vocab_dict['UNK']] += 1

        bow = calculate_tfidf(inputs, vocab_dict)

        inputs = torch.tensor(bow, dtype=torch.float32).to(device)
        labels = test_labels[i:i + batch_size]
        labels = torch.tensor(labels, dtype=torch.long).to(device)

        outputs = model(inputs)

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Collect predictions and true labels
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

    # Calculate accuracy
    accuracy = 100 * correct / total
    print(f'Accuracy of the network on the {total} test inputs: {accuracy} %')

    # Calculate confusion matrix, precision, recall, and F1 scores
    confusion = confusion_matrix(true_labels, predictions)
    classification_rep = classification_report(true_labels, predictions, target_names=[str(i) for i in range(14)])
    f1_micro = f1_score(true_labels, predictions, average='micro')
    f1_macro = f1_score(true_labels, predictions, average='macro')
    f1_weighted = f1_score(true_labels, predictions, average='weighted')
    precision = precision_score(true_labels, predictions, average='weighted')
    recall = recall_score(true_labels, predictions, average='weighted')

    print("Confusion Matrix:")
    print(confusion)
    print("Classification Report:")
    print(classification_rep)
    # print("F1 Micro:", f1_micro)
    # print("F1 Macro:", f1_macro)
    # print("F1 Weighted:", f1_weighted)
    # print("Precision:", precision)
    # print("Recall:", recall)


Accuracy of the network on the 70000 test inputs: 96.12571428571428 %
Confusion Matrix:
[[4555   50   46   11   24   62   65   10    1    0    5   55   29   87]
 [  40 4877   13    1   17    6   31    4    2    0    1    0    4    4]
 [  36    8 4570   12   83    1   11    0    1    0    1  104   33  140]
 [   3    1   51 4909   20    2    4    0    0    2    1    1    5    1]
 [  15   15   84   15 4842    4    7    0    1    1    0    0    5   11]
 [  45    4    2    2    2 4912   10    5    0    3    2    3    7    3]
 [  56   65   11    4   16   16 4783   31    4    1    0    3    3    7]
 [   4    2    2    1    4    3   31 4935    9    3    3    0    1    2]
 [   4    9    4    1    7    0   30   58 4880    3    0    0    2    2]
 [   2    5    9    3    5    1    1   14    2 4809  141    0    3    5]
 [  13    2    4    0    1    2    2   12    0   87 4869    1    2    5]
 [  13    3   60    4    4    2    6    3    0    2    0 4868   21   14]
 [  10    4   27    9    5   10    9